# Shark Attacks

## 1. Import modules

In [1]:
import numpy as np
import pandas as pd
import re

import warnings

warnings.filterwarnings('ignore')


# Some Resources:
# https://github.com/AnaAGG/workshop_regex/blob/main/3-%20Regex%20in%20pandas.ipynb
# https://stackoverflow.com/questions/39475978/apply-function-to-each-cell-in-dataframe
# https://towardsdatascience.com/data-cleaning-using-python-pandas-f6fadc433535
# https://towardsdatascience.com/10-tricks-for-converting-numbers-and-strings-to-datetime-in-pandas-82a4645fc23d

In [2]:
# !kaggle datasets download -d teajay/global-shark-attacks

In [3]:
# !ls

In [4]:
# #descomprimimos el .zip 
# !tar -xzvf global-shark-attacks.zip

In [5]:
# !ls

In [6]:
# #borramos el zip ya que lo hemos descomprimido y ya tenemos el csv
# !rm -rf global-shark-attacks.zip

## 2. Import CSV

In [7]:
# Read and convert the csv source data into a pandas dataframe.

pd.set_option('display.max_columns', None)
sharks_attacks = pd.read_csv("attacks.csv",encoding = "ISO-8859-1")

## 3. Exploring Data

### 3.1 First overlook

In [8]:
# Shows first 5(default) rows of the dataset.

sharks_attacks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [9]:
# Generate various summary statistics, excluding NaN values.

sharks_attacks.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,6300.0,1927.272381,281.116308,0.0,1942.0,1977.0,2005.0,2018.0
original order,6309.0,3155.999683,1821.396206,2.0,1579.0,3156.0,4733.0,6310.0


This output doesn't say much (totally useless, pandas is recognizing years and original order as continuous variables). It seems the mayority of the variables in this dataset are discrete/categorical type. We might be able to use "count" methods for these ones after applying some data cleaning and data consolidation.
We will get clearer information later on.

In [10]:
# Prints a concise summary of a DataFrame.

sharks_attacks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [11]:
# Gets number of registers and variables of this dataset

sharks_attacks.shape

(25723, 24)

In [12]:
# Calculates the percentage of null registers for each variable

percent_missing = round(sharks_attacks.isnull().sum() * 100 / len(sharks_attacks), 2)
percent_missing

Case Number                66.17
Date                       75.50
Year                       75.51
Type                       75.52
Country                    75.69
Area                       77.27
Location                   77.60
Activity                   77.62
Name                       76.32
Sex                        77.70
Age                        86.51
Injury                     75.61
Fatal (Y/N)                77.60
Time                       88.54
Species                    86.53
Investigator or Source     75.57
pdf                        75.50
href formula               75.50
href                       75.50
Case Number.1              75.50
Case Number.2              75.50
original order             75.47
Unnamed: 22               100.00
Unnamed: 23                99.99
dtype: float64

It looks like the mayority of values for each variables available is missing

In [13]:
# Gets column names

column_names = list(sharks_attacks.columns.values)
column_names

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ',
 'Investigator or Source',
 'pdf',
 'href formula',
 'href',
 'Case Number.1',
 'Case Number.2',
 'original order',
 'Unnamed: 22',
 'Unnamed: 23']

It seems these column names don't follow a proper name convention and some of them have spaces before and after

In [14]:
# Registers consistency of each variable

for cols in column_names:
    uniques = sharks_attacks[cols].unique()
    print(cols)
    print(uniques)
    
    print('\n' * 3)



Case Number
['2018.06.25' '2018.06.18' '2018.06.09' ... 'ND.0001' '0' 'xx']




Date
['25-Jun-2018' '18-Jun-2018' '09-Jun-2018' ... '1883-1889' '1845-1853' nan]




Year
[2018. 2017.   nan 2016. 2015. 2014. 2013. 2012. 2011. 2010. 2009. 2008.
 2007. 2006. 2005. 2004. 2003. 2002. 2001. 2000. 1999. 1998. 1997. 1996.
 1995. 1984. 1994. 1993. 1992. 1991. 1990. 1989. 1969. 1988. 1987. 1986.
 1985. 1983. 1982. 1981. 1980. 1979. 1978. 1977. 1976. 1975. 1974. 1973.
 1972. 1971. 1970. 1968. 1967. 1966. 1965. 1964. 1963. 1962. 1961. 1960.
 1959. 1958. 1957. 1956. 1955. 1954. 1953. 1952. 1951. 1950. 1949. 1948.
 1848. 1947. 1946. 1945. 1944. 1943. 1942. 1941. 1940. 1939. 1938. 1937.
 1936. 1935. 1934. 1933. 1932. 1931. 1930. 1929. 1928. 1927. 1926. 1925.
 1924. 1923. 1922. 1921. 1920. 1919. 1918. 1917. 1916. 1915. 1914. 1913.
 1912. 1911. 1910. 1909. 1908. 1907. 1906. 1905. 1904. 1903. 1902. 1901.
 1900. 1899. 1898. 1897. 1896. 1895. 1894. 1893. 1892. 1891. 1890. 1889.
 1888. 1887. 1886. 1885. 18

In [15]:
# Gets unique values per column

sharks_attacks_unique = pd.DataFrame(sharks_attacks.nunique(), columns = ['count'])
sharks_attacks_unique

,count
Case Number,6287
Date,5433
Year,249
Type,8
Country,212
Area,825
Location,4108
Activity,1532
Name,5230
Sex,6


## 4. Cleaning the dataset

### 4.1 Columns name convention

In [16]:
# This function (nameconvention) changes the default columns names into a proper name convention style (snake_case)

import sharks_attacks_functions as saf

In [17]:
# Renaming columns...

# 1 - Replaces each column_names list items using nameconvention function
column_names = saf.nameconvention(column_names)

# 2 - Renames our dataset columns with our new column names
sharks_attacks = sharks_attacks.rename(columns=column_names)

sharks_attacks.head(0)

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23


### 4.2 Remove extra spaces (entire dataset)

In [18]:
# Removes extra spaces before and after each value of each column so it's easier to aggregate unique discrete values

sharks_attacks2 = sharks_attacks.applymap(lambda x: str(x).strip() if isinstance(x, str) else x)
sharks_attacks2.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [19]:
# We compare if we have reduce unique discrete values 

sharks_attacks_unique = pd.DataFrame(sharks_attacks.nunique(), columns = ['count'])
sharks_attacks_unique2 = pd.DataFrame(sharks_attacks2.nunique(), columns = ['count'])

sharks_attacks_unique2.compare(sharks_attacks_unique)

count        
                          self   other
date                    5426.0  5433.0
country                  204.0   212.0
area                     801.0   825.0
location                4065.0  4108.0
activity                1504.0  1532.0
name                    5224.0  5230.0
sex                        5.0     6.0
age                      149.0   157.0
injury                  3671.0  3737.0
fatal_(y/n)                6.0     8.0
time                     359.0   366.0
species                 1467.0  1549.0
investigator_or_source  4927.0  4969.0

### 4.3 Find duplicate columns

#### We will try two find if it exists two or more exact columns with the same values

In [20]:
saf.get_dup_columns(sharks_attacks2)

[]

There is no complitely duplicate columns.

#### Let's check if with a function we can found possible very similar columns.
#### Create a function to detect percentage of similitud among 2 columns

#### Make a table comparing column to column

In [21]:
compared_cols = pd.DataFrame()
compared_cols['index'] = sharks_attacks2.columns
compared_cols = compared_cols.set_index('index')
for col1 in sharks_attacks2.columns:
    column = list()
    for col2 in sharks_attacks2.columns:
        column.append(saf.compare_rows(sharks_attacks2, col1, col2))
    compared_cols[col1] = column

In [22]:
compared_cols

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
index,,,,,,,,,,,,,,,,,,,,,,,,
case_number,100.00,0.10,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.00,99.62,99.94,0.00,0.0,0.0
date,0.07,100.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.11,0.10,0.00,0.0,0.0
year,0.00,0.00,100.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.02,0.0,0.0
type,0.00,0.00,0.00,100.00,0.0,0.00,0.00,0.24,0.00,0.00,0.0,0.00,0.00,0.00,2.97,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
country,0.00,0.00,0.00,0.00,100.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
area,0.00,0.00,0.00,0.00,0.0,100.00,0.19,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
location,0.00,0.00,0.00,0.00,0.0,0.19,100.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
activity,0.00,0.00,0.00,0.22,0.0,0.00,0.00,100.00,0.00,0.02,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
name,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,100.00,0.02,0.0,0.02,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


#### It seems that "case_number" is very similar to "case_number.1" and "case_sumber.2" 
#### href and href formula are very much alike.

### 4.4 Column Date

#### Missing or incomplete values

#### Let's check if there's information within the rest of the columns where "date" has null values and in that case, we'll see if we can get the missing values of "date" from other columns

In [23]:
sharks_attacks2[sharks_attacks2["date"].isnull()].count()

case_number               2401
date                         0
year                         0
type                         0
country                      0
area                         0
location                     0
activity                     0
name                         0
sex                          0
age                          0
injury                       0
fatal_(y/n)                  0
time                         0
species                      0
investigator_or_source       0
pdf                          0
href_formula                 0
href                         0
case_number.1                0
case_number.2                0
original_order               7
unnamed:_22                  0
unnamed:_23                  0
dtype: int64

#### Nothing to scrape here and it seems we won't miss any relevant data if we get rid of rows where "date" has null values

In [24]:
sharks_attacks3 = sharks_attacks2[pd.notnull(sharks_attacks2['date'])]

In [25]:
sharks_attacks3[sharks_attacks3["date"].isnull()]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23


In [26]:
sharks_attacks2.shape

(25723, 24)

In [27]:
sharks_attacks3.shape

(6302, 24)

#### Consistency

#### It seems values within column "date" sometimes don't have any consistency. Rather than trying to fix all the unformatted values within this column, let's see if we can get than information from other columns.
#### We saw the columns case_number, case_number.1 and case_number.2 were very much alike and contained the date in its values 

#### Before we pick which one of those columns we are going to use to extract the dates. Let's see the differencies between case_number, case_number.1 and case_number.2

In [28]:
sharks_attacks3[['date','year','sex','case_number','case_number.1','case_number.2']].loc[(sharks_attacks3['case_number'] != sharks_attacks3['case_number.1']) | (sharks_attacks3['case_number'] != sharks_attacks3['case_number.2']) | (sharks_attacks3['case_number.1'] != sharks_attacks3['case_number.2'])]

,date,year,sex,case_number,case_number.1,case_number.2
34,03-Apr-2018,2018.0,M,2018.04.03,2018.04.02,2018.04.03
117,20-Jul-2017,2017.0,M,2017.07.20.a,2017/07.20.a,2017.07.20.a
144,05-May-2017,2017.0,M,2017.05.06,2017.06.06,2017.05.06
217,16-Sep-2016,2016.0,M,2016.09.15,2016.09.16,2016.09.15
314,24-Jan-2016,2016.0,M,2016.01.24.b,2015.01.24.b,2016.01.24.b
334,07-Nov-2015,2015.0,F,2015.12.23,2015.11.07,2015.12.23
339,28-Oct-2015,2015.0,M,2015.10.28.a,2015.10.28,2015.10.28.a
390,10-Jul-2015,2015.0,M,2015.07-10,2015.07.10,2015.07.10
560,04-May-2014,2014.0,NaN,2014.05.04,2013.05.04,2014.05.04
3522,05-Jul-1967,1967.0,M,1967.07.05,1967/07.05,1967.07.05


#### It looks like case_number.1 and case_number.2 are corrections of case_number

#### The rule for the naming in these three columns seems to be  year.month.day.victim.Reported yyyy.mm.dd.[a].[R]
#### I see some punctuations types I'm going to correct first for the three of them


In [29]:
sharks_attacks3[['case_number', 'case_number.1', 'case_number.2']] = sharks_attacks3[['case_number', 'case_number.1', 'case_number.2']].astype(str)

In [30]:
sharks_attacks3['case_number']= sharks_attacks3['case_number'].str.replace(',', '.')

In [31]:
sharks_attacks3['case_number.1']= sharks_attacks3['case_number.1'].str.replace(',', '.')

In [32]:
sharks_attacks3['case_number.2']= sharks_attacks3['case_number.2'].str.replace(',', '.')

#### case_number.2 seems to be last updated, so we're are going to replace the values where case_number and case_number.2  differ into case_number 

In [33]:
# Checks values where case_number and case_number.2 differ

sharks_attacks3[['date','year','sex','case_number','case_number.2']].loc[(sharks_attacks3['case_number'] != sharks_attacks3['case_number.2'])]

,date,year,sex,case_number,case_number.2
390,10-Jul-2015,2015.0,M,2015.07-10,2015.07.10
4949,Reported 08-Feb-1934,1934.0,M,1934.01.08.R,1934.02.08.R
5488,Reported 06-Sep-1905,1905.0,M,nan,1905.09.06.R
5944,May-1864,1864.0,M,1864.05,1864.05.00


In [34]:
# Replacement of values 
sharks_attacks3['case_number'].iloc[[390, 4949, 5488, 4944]] = sharks_attacks3['case_number.2'].iloc[[390, 4949, 5488, 4944]]

In [35]:
sharks_attacks3[sharks_attacks3["case_number"].isnull()].count()

case_number               0
date                      0
year                      0
type                      0
country                   0
area                      0
location                  0
activity                  0
name                      0
sex                       0
age                       0
injury                    0
fatal_(y/n)               0
time                      0
species                   0
investigator_or_source    0
pdf                       0
href_formula              0
href                      0
case_number.1             0
case_number.2             0
original_order            0
unnamed:_22               0
unnamed:_23               0
dtype: int64

In [36]:
# We make sure there're not more punctuation typos within "case_numer"

sharks_attacks3['case_number']= sharks_attacks3['case_number'].str.replace('-', '.')
sharks_attacks3['case_number']= sharks_attacks3['case_number'].str.replace('/', '.')

#### Here we are trying to get as much correct date registers from "case_number" as possible

In [37]:
# We check how many registers have valid date format within "case_number"

sharks_attacks3[sharks_attacks3["case_number"].str.match("^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/)([1-9]|0[1-9]|1[0-2])(\.|-|/)([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$") == True]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,1900.09.05,05-Sep-1900,1900.0,Unprovoked,USA,Hawaii,"Waikiki Beach, Oahu",Floating,Joe Hartman,M,NaN,"Bathing suit torn & ""imprints of the shark's t...",N,Afternoon,NaN,"Honolulu Republican, 9/6/1900",1900.09.05-Hartman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1900.09.05,1900.09.05,747.0,NaN,NaN
5557,1900.08.21,21-Aug-1900,1900.0,Unprovoked,USA,North Carolina,"Southport, Brunswick County",Bathing,Burris,M,NaN,Left hand lacerated,N,Afternoon,NaN,"C. Creswell, GSAF",1900.08.21-Burriss.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1900.08.21,1900.08.21,746.0,NaN,NaN
5558,1900.07.31,31-Jul-1900,1900.0,Unprovoked,CROATIA,Primorje-Gorski Kotar County,"Volosko, Opatija",Swimming,male,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1900.07.31-Croatia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1900.07.31,1900.07.31,745.0,NaN,NaN
5559,1900.07.14,14-Jul-1900,1900.0,Invalid,USA,Hawaii,"Makapu'u Point, O'ahu",Hunting seashells,Emil Uhlbrecht & unidentified person,M,NaN,"Believed drowned. Uhlbrechts foot, and the pe...",NaN,NaN,Questionable,"Los Angeles Times, 7/28/1900",1900.07.14-Uhlbrecht.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1900.07.14,1900.07.14,744.0,NaN,NaN


In [38]:
# We check the opposite here in case we can make some data cleaning 

sharks_attacks3[sharks_attacks3["case_number"].str.match("^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/)([1-9]|0[1-9]|1[0-2])(\.|-|/)([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$") != True]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN
12,2018.05.13.b,13-May-2018,2018.0,Unprovoked,USA,South Carolina,"Hilton Head Island, Beaufort County",Swimming,Jei Turrell,M,10,Severe bite to right forearm,N,15h00,NaN,"C. Creswell, GSAF & K. McMurray TrackingSharks...",2018.05.13.b-Turrell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.13.b,2018.05.13.b,6291.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN


#### We see within the "case_column" we could get rid of register with extra characters at the end (e.g.  2018.05.13.b --> 2018.05.13)

In [39]:
def re_sub(x):
    x == str(x)
    y = re.sub(".[aA-zZ]$", "", x)
    return y

In [40]:
sharks_attacks3["new_date"] = sharks_attacks3["case_number"].apply(re_sub)

In [41]:
sharks_attacks3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             6302 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null   

In [42]:
# We check again if the number of registers within "case_number" with a 0000.00.00 format has increase

sharks_attacks3[sharks_attacks3["new_date"].str.match("^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/)([1-9]|0[1-9]|1[0-2])(\.|-|/)([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$") == True].shape

(4931, 25)

#### It has increased, from 3486 rows to 4931

#### We've seen that there're some registers that don't have the correct case_number format. For these confirmation dates are not very precisely. So we are going to delete them

In [43]:
sharks_attacks4 = sharks_attacks3[sharks_attacks3["new_date"].str.match("^ND.\d+") != True]

#### Let's see if we can do some more cleaning by finding registers in case_numbers with character length greater than 10 ( yyyy.mm.dd)

In [44]:
sharks_attacks4[sharks_attacks4["new_date"].str.len() > 10]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23,new_date
847,2011.10.29.,29-Oct-2011,2011.0,Unprovoked,USA,California,"Marina State Beach, Monterey County",Surfing,Eric Tarantino,M,27,"Lacerations to right wrist, foream & neck",N,NaN,White shark,R. Collier,2011.10.29-Tarantino.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2011.10.29.,2011.10.29.,5456.0,NaN,NaN,2011.10.29.
849,2011.10.28.,29-Oct-2011,2011.0,Provoked,SOUTH AFRICA,KwaZulu-Natal,"uShaka Aquarium, Durban",Diving,NaN,NaN,NaN,Arm bitten by captive shark PROVOKED INCIDENT,N,NaN,Raggedtooth shark,"Durban Radio, 10/29/2011",2011.10.28-uShaka-Aquarium.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2011.10.28.,2011.10.28.,5454.0,NaN,NaN,2011.10.28.
881,2011.08.17.c.,17-Aug-2011,2011.0,Unprovoked,USA,North Carolina,"Kure Beach, New Hanover County",Wading,Trang Aronian,F,20s,Lacerations to foot,N,17h00,Possibly a 5' to 6' sandtiger shark,C. Creswell,2011.08.17.c-Aronian.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2011.08.17.c.,2011.08.17.c.,5422.0,NaN,NaN,2011.08.17.c.
882,2011.08.17.b.,17-Aug-2011,2011.0,Invalid,USA,North Carolina,"Wrightsville Beach, New Hanover County",NaN,NaN,M,12,Abrasions to left hand,NaN,16h00,Shark involvement not confirmed,"C. Creswell, GSAF; Wway, 8/17/2011",2011.08.17.b-child.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2011.08.17.b.,2011.08.17.b.,5421.0,NaN,NaN,2011.08.17.b.
1135,2009.04.17.,17-Apr-2009,2009.0,Unprovoked,USA,Florida,"Walton Rocks, St Lucie County",Surfing,Alexander Wagner,M,31,Laceration to forearm,N,13h15,2' to 3' shark,"TC Palm, 4/17/09",2009.04.17-Wagner.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2009.04.17.,2009.04.17.,5168.0,NaN,NaN,2009.04.17.
1346,2007.08.19.a.,19-Aug-2007,2007.0,Unprovoked,USA,South Carolina,"Lakewood Campground, Grand Strand, Horry County",Playing,male,M,7,Right calf bitten,N,10h00,NaN,"C. Creswell, GSAF",2007.08.19.a-boy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2007.08.19.a.,2007.08.19.a.,4957.0,NaN,NaN,2007.08.19.a.
1849,2002.06.13.R2,Reported 13-Jun-2002,2002.0,Unprovoked,PAPUA NEW GUINEA,Louisiade Archipelago,"Brooker Island , Calvados Chain",NaN,NaN,NaN,NaN,"Arm severely lacerated, surgically amputated",N,NaN,NaN,"PNG Post-Courier, 6/13/2002, p.5",2002.06.13-R.2-beche-de-mer.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2002.06.13.R2,2002.06.13.R2,4454.0,NaN,NaN,2002.06.13.R2
1850,2002.06.13.R1,Reported 13-Jun-2002,2002.0,Unprovoked,PAPUA NEW GUINEA,Louisiade Archipelago,"Gawa Reefs, Sudest Island",Attempting to retreive a dinghy,an elementary school teacher,NaN,NaN,FATAL,Y,NaN,NaN,"PNG Post-Courier, 6/13/2002, p.5",2002.06.13.R.1-schoolteacher.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2002.06.13.R1,2002.06.13.R1,4453.0,NaN,NaN,2002.06.13.R1
2276,1996.10.05.,05-Oct-1996,1996.0,Unprovoked,USA,California,"Dillon Beach, Marin County",Surfing,Mark Quirt,M,22,Lower leg bitten,N,13h00,5.5 m to 6 m [18' to 20'] white shark,"R. Collier, pp.156-157",1996.10.05-Quirt_Collier.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1996.10.05.,1996.10.05.,4027.0,NaN,NaN,1996.10.05.
2340,1995.09.19.,19-Sep-1995,1995.0,Unprovoked,USA,Florida,"Matanzas Bay Inlet, St. Johns County",Surfing,Gavin Korth,M,19,Left arm & hand lacerated,N,NaN,1.8 m [6'] shark,"Orlando Sentinel, 9/20/1995, p.C.3",1995.09.19-Korth.pdf,http://sharkattackfile

In [45]:
# The fastest way to delete those extra characters is limiting the length of the strings to 10 chars

sharks_attacks4["new_date"]= sharks_attacks4["new_date"].str[0:10]

#### We filter out those values in "new_date" that start with 0 which means we can't get any date information about those

In [46]:
sharks_attacks4 = sharks_attacks4[sharks_attacks4["new_date"].str.match("^0") != True]

#### Finally we are only going to keep those register within "new_date" with a complete date format yyyy.mm.dd (eg. not accepted 2018.09.00)

In [47]:
sharks_attacks5 = sharks_attacks4[sharks_attacks4["new_date"].str.match("^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/)([1-9]|0[1-9]|1[0-2])(\.|-|/)([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$") == True]

#### Finally we've decided we are going to use the column "new_date" (cleaned "case_number") as a date column 

In [48]:
sharks_attacks5["new_date"] = pd.to_datetime(sharks_attacks5["new_date"], format='%Y.%m.%d')

#### For analysis purposes we are going to create another column with just  year and month

In [49]:
sharks_attacks5["year"] = pd.DatetimeIndex(sharks_attacks5["new_date"]).year
sharks_attacks5["month_year"] = pd.to_datetime(sharks_attacks5["new_date"]).dt.to_period('M')
sharks_attacks5.shape

(4967, 26)

In [50]:
sharks_attacks5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   case_number             4967 non-null   object        
 1   date                    4967 non-null   object        
 2   year                    4967 non-null   int64         
 3   type                    4964 non-null   object        
 4   country                 4954 non-null   object        
 5   area                    4733 non-null   object        
 6   location                4697 non-null   object        
 7   activity                4630 non-null   object        
 8   name                    4839 non-null   object        
 9   sex                     4573 non-null   object        
 10  age                     3234 non-null   object        
 11  injury                  4953 non-null   object        
 12  fatal_(y/n)             4556 non-null   object  

In [51]:
sharks_attacks5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   case_number             4967 non-null   object        
 1   date                    4967 non-null   object        
 2   year                    4967 non-null   int64         
 3   type                    4964 non-null   object        
 4   country                 4954 non-null   object        
 5   area                    4733 non-null   object        
 6   location                4697 non-null   object        
 7   activity                4630 non-null   object        
 8   name                    4839 non-null   object        
 9   sex                     4573 non-null   object        
 10  age                     3234 non-null   object        
 11  injury                  4953 non-null   object        
 12  fatal_(y/n)             4556 non-null   object  

### 4.4 Drop columns

#### "original order" column seems the primitive index.  We are goin to keep the index of the dataframe as it is so we don't need this columnn

In [52]:
sharks_attacks5 = sharks_attacks5.drop(columns = ['original_order'])

#### Now I can drop the columns case_number.1 and case-number.2. We merged them into "case_number" before

In [53]:
sharks_attacks5 = sharks_attacks5.drop(['case_number.1', 'case_number.2'], axis=1)

#### The cols 'Unnamed: 22' and 'Unnamed: 23' have only one value and two values. They don't give us further information, we can delete them


In [54]:
sharks_attacks5 = sharks_attacks5.drop(['unnamed:_22', 'unnamed:_23','investigator_or_source','pdf', 'href_formula','href'], axis=1)

In [55]:
sharks_attacks5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   case_number  4967 non-null   object        
 1   date         4967 non-null   object        
 2   year         4967 non-null   int64         
 3   type         4964 non-null   object        
 4   country      4954 non-null   object        
 5   area         4733 non-null   object        
 6   location     4697 non-null   object        
 7   activity     4630 non-null   object        
 8   name         4839 non-null   object        
 9   sex          4573 non-null   object        
 10  age          3234 non-null   object        
 11  injury       4953 non-null   object        
 12  fatal_(y/n)  4556 non-null   object        
 13  time         2779 non-null   object        
 14  species      3010 non-null   object        
 15  new_date     4967 non-null   datetime64[ns]
 16  month_

### 4.5 Column time

#### We check format and unique values from "time" column

In [56]:
sharks_attacks5.time.unique()

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       'Midday', '13h30', '10h45', '11h20', '11h45', '19h30', '08h30',
       '15h45', 'Shortly before 12h00', '17h34', '17h10', '11h15',
       '08h50', '17h45', '13h00', '10h20', '13h20', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', '15h49', '19h00', 'Midnight',
       '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50', '13h50',
       '19h20', '10h25', '10h45-11h15', '16h45', '15h52', '06h15',
       '14h40', '19h45'

#### We replace "h" within "time" values because it's easier to change the datatype into datetime this way

In [57]:
sharks_attacks5["time"]= sharks_attacks5['time'].str.replace("h", ":")
sharks_attacks5["time"]= sharks_attacks5['time'].str.replace("j", ":")
sharks_attacks5["time"]= sharks_attacks5['time'].str.replace(">", "")

#### We check who many values from "time column doesn't follow the Hours and minutes, 24-hour clock format and it's not null

In [58]:
clean_time = sharks_attacks5[(sharks_attacks5["time"].str.match("^(2[0-3]|[01]?[0-9]):([0-5]?[0-9])$") != True) & sharks_attacks5["time"].notnull()]

In [59]:
clean_time.time.unique()

array(['14:00  -15:00', 'Late afternoon', 'Morning', 'Afternoon',
       '19:00, Dusk', 'Nig:t', 'Midday', 'S:ortly before 12:00',
       'After noon', '1300', '14:30 / 15:30', 'Midnig:t', '09:30 / 10:00',
       '10:45-11:15', 'Evening', 'Sometime between 06:00 & 08:oo',
       'Early afternoon', '07:00 - 08:00', '18:15-18:30', '09:00 - 09:30',
       '0830', 'Just before noon', '1600', 'Early morning', 'Dawn', 'AM',
       'A.M.', 'Dusk', 'Lunc:time', '0500', 'Before 07:00',
       '10:00 -- 11:00', '"Just before 11:00"', '11:115', 'Sunset', '',
       '"Evening"', 'Just before sundown', 'Between 05:00 and 08:00',
       '17:00 or 17:40', '--', 'Just after 12:00', 'Early Morning',
       'S:ortly after midnig:t', '13:345', '09:00 -10:00',
       '20:45 (Sunset)', 'Late morning', 'P.M.', 'S:ortly before 13:00',
       '8:04 pm', 'Possibly same incident as 2000.08.21', 'After Dusk',
       'Noon', '2 :ours after Opperman', 'Mid afternoon', 'Mid morning',
       '11:00 / 11:30', '"Nig:t

#### too many exceptions to clean and we can't delete them as the rest of the columns have notnul values in it. Will keep these ones as unknown 

In [60]:
sharks_attacks5[sharks_attacks5["time"].isnull()].count()

case_number    2188
date           2188
year           2188
type           2186
country        2180
area           2008
location       1972
activity       1911
name           2083
sex            1889
age             956
injury         2178
fatal_(y/n)    1906
time              0
species        1172
new_date       2188
month_year     2188
dtype: int64

In [61]:
# sharks_attacks5["time"] = pd.to_datetime(sharks_attacks5["time"], format = "%H:%M", errors="coerce")

In [62]:
# sharks_attacks5.info()

### 4.5 Column sex

#### Check categories within "sex" column

In [63]:
sharks_attacks5.sex.unique()

array(['F', 'M', nan, 'lli', 'N', '.'], dtype=object)

#### We replace everything that is not defined as "F" or "M" for a "U". "U" stands for "Unknown"

In [64]:
sharks_attacks5["sex"]= sharks_attacks5['sex'].str.replace("lli", "U")
sharks_attacks5["sex"]= sharks_attacks5['sex'].str.replace("N", "U")
sharks_attacks5["sex"]= sharks_attacks5['sex'].str.replace(".", "U")

In [65]:
sharks_attacks5.fillna(value={"sex": "U"}, inplace=True)

In [66]:
# Cast types to save memory.
sharks_attacks5["sex"] = sharks_attacks5["sex"].astype("category")

In [67]:
sharks_attacks5.sex.unique()

['F', 'M', 'U']
Categories (3, object): ['F', 'M', 'U']

In [68]:
# Renaming DataFrame categories
sharks_attacks5["sex"].cat.rename_categories(new_categories={"M": "Male", "F": "Female", "U": "Unknown"}, inplace=True)

In [69]:
sharks_attacks5["sex"].value_counts()

Male       4015
Female      555
Unknown     397
Name: sex, dtype: int64

### 4.5 Column age

#### Check unique values within "age" column. This column has a lot of missing values so will try to fix as much values as possible

In [70]:
sharks_attacks5.age.unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '30', '60', '33', '29', '54', '34', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '', '30 or 36', '6½', '21 & ?', '75', '33 or 37',
       'mid-30s', '23 & 20', '7      &    31', '20?', '32 & 30',
       '16 to 18', '87', '67', 'Elderly', 'mid-20s', '21 or 26', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '(adult)', '33 & 37',
       '25 or 28', '37, 67, 35, 27,  ? & 27', '21, 34,24 & 35', '30 & 32',

In [71]:
sharks_attacks5["age"]= sharks_attacks5["age"].str.replace("s", "")
sharks_attacks5["age"]= sharks_attacks5["age"].str.replace("6½", "6")
sharks_attacks5["age"]= sharks_attacks5["age"].str.replace("mid-", "")


In [72]:
sharks_attacks5[(sharks_attacks5["age"].str.match("[1-9]{1,2}") != True) & sharks_attacks5["age"].notnull()]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year
182,2017.01.22,22-Jan-2017,2017,Unprovoked,USA,Florida,"Vero Beach, Indian River County",NaN,male,Male,Teen,Puncture wounds to lower arm or hand,N,14:00,NaN,2017-01-22,2017-01
294,2016.03.28.b,28-Mar-2016,2016,Unprovoked,USA,Florida,"Fort Myers Beach, Lee County",NaN,Nick Kawa,Male,Teen,Minor injury to arm. Possibly caused by smalll...,NaN,NaN,Shark involvement not confirmed,2016-03-28,2016-03
526,2014.07.27,27-Jul-2014,2014,Unprovoked,USA,North Carolina,"Sunset Beach, Brunswick County",Swimming,male,Male,Teen,Left foot bitten,N,NaN,Possibly juvenile tiger shark,2014-07-27,2014-07
567,2014.04.04.b,04-Apr-2014,2014,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,Male,teen,Minor puncture wounds to lower left leg,N,13:50,NaN,2014-04-04,2014-04
568,2014.04.04.a,04-Apr-2014,2014,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,Male,teen,Lacerations to foot,N,13:30,NaN,2014-04-04,2014-04
653,2013.07.17.R,Reported 17-Jul-2013,2013,Unprovoked,USA,Florida,"Butler Beach, St Augustine, St. Johns Co...",Swimming,male,Male,teen,4 cuts to posterior calf,N,NaN,NaN,2013-07-17,2013-07
983,2010.10.01,01-Oct-2010,2010,Unprovoked,SOUTH AFRICA,Western Cape Province,"Melkbaai, Strand",Surfing,male,Male,Teen,3 lacerations to foot,N,14:30,NaN,2010-10-01,2010-10
1303,2007.12.15,15-Dec-2007,2007,Unprovoked,AUSTRALIA,Queensland,South Stradbroke Island,Swimming,Josh Edwards,Male,teen,Lacerations to hand,N,NaN,"""a small shark""",2007-12-15,2007-12
1331,2007.09.16.a,16-Sep-2007,2007,Unprovoked,USA,Florida,"Flagler Beach, Flagler County",Surfing,Jessica Riley,Female,Teen,"No injury, surfboard bitten",N,Early morning,9.5' shark?,2007-09-16,2007-09
1520,2005.11.29.R,Reported 29-Nov-2005,2005,Unprovoked,USA,Florida,"Cape San Blas, Gulf County",Surfing,John Larsen,Male,teen,Left foot bitten,N,NaN,NaN,2005-11-29,2005-11


#### After some data cleaning within the "age" column we are going to turn an "Unknown" category everything that doesn't follow 1 or two digits pattern

In [73]:
sharks_attacks5.loc[sharks_attacks5["age"].str.match("[1-9]{1,2}") != True, "age"] = "Unknown"
sharks_attacks5

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,Female,57,"No injury to occupant, outrigger canoe and pad...",N,18:00,White shark,2018-06-25,2018-06
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,Female,11,Minor injury to left thigh,N,14:00 -15:00,NaN,2018-06-18,2018-06
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,Male,48,Injury to left lower leg from surfboard skeg,N,07:45,NaN,2018-06-09,2018-06
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,Male,Unknown,Minor injury to lower leg,N,NaN,2 m shark,2018-06-08,2018-06
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,Male,Unknown,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",2018-06-04,2018-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,1900.09.05,05-Sep-1900,1900,Unprovoked,USA,Hawaii,"Waikiki Beach, Oahu",Floating,Joe Hartman,Male,Unknown,"Bathing suit torn & ""imprints of the shark's t...",N,Afternoon,NaN,1900-09-05,1900-09
5557,1900.08.21,21-Aug-1900,1900,Unprovoked,USA,North Carolina,"Southport, Brunswick County",Bathing,Burris,Male,Unknown,Left hand lacerated,N,Afternoon,NaN,1900-08-21,1900-08
5558,1900.07.31,31-Jul-1900,1900,Unprovoked,CROATIA,Primorje-Gorski Kotar County,"Volosko, Opatija",Swimming,male,Male,Unknown,FATAL,Y,NaN,NaN,1900-07-31,1900-07
5559,1900.07.14,14-Jul-1900,1900,Invalid,USA,Hawaii,"Makapu'u Point, O'ahu",Hunting seashells,Emil Uhlbrecht & unidentified person,Male,Unknown,"Believed drowned. Uhlbrechts foot, and the pe...",NaN,NaN,Questionable,1900-07-14,1900-07


In [74]:
sharks_attacks5[sharks_attacks5["age"].isnull()]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year


#### Change column type to integer

In [75]:
sharks_attacks5["age"] = pd.to_numeric(sharks_attacks5["age"], errors='coerce')

### 4.6 Column type

#### Checking unique categories within "type" column

In [76]:
sharks_attacks5.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

#### We correct some misspellings for "Boating" category

In [77]:
sharks_attacks5["type"]= sharks_attacks5['type'].str.replace("^Boat$", "Boating")
sharks_attacks5["type"]= sharks_attacks5['type'].str.replace("Boatomg", "Boating")

In [78]:
sharks_attacks5.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan], dtype=object)

In [79]:
sharks_attacks5.fillna(value={"type": "Unknown"}, inplace=True)

In [80]:
sharks_attacks5.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', 'Unknown'], dtype=object)

In [81]:
# Cast types to save memory.
sharks_attacks5["type"] = sharks_attacks5["type"].astype("category")

In [82]:
sharks_attacks5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   case_number  4967 non-null   object        
 1   date         4967 non-null   object        
 2   year         4967 non-null   int64         
 3   type         4967 non-null   category      
 4   country      4954 non-null   object        
 5   area         4733 non-null   object        
 6   location     4697 non-null   object        
 7   activity     4630 non-null   object        
 8   name         4839 non-null   object        
 9   sex          4967 non-null   category      
 10  age          3169 non-null   float64       
 11  injury       4953 non-null   object        
 12  fatal_(y/n)  4556 non-null   object        
 13  time         2779 non-null   object        
 14  species      3010 non-null   object        
 15  new_date     4967 non-null   datetime64[ns]
 16  month_

### 4.7 Column fatal

#### Checking unique categories within "fatal" column

In [83]:
# Checkpoint
sharks_attacks6 = sharks_attacks5

In [84]:
sharks_attacks6["fatal_(y/n)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017'], dtype=object)

#### Checking how many register we have of each category

In [85]:
shark_count_fatal = sharks_attacks6[["fatal_(y/n)"]]
shark_count_fatal.groupby(["fatal_(y/n)"])["fatal_(y/n)"].count()

fatal_(y/n)
2017          1
M             1
N          3646
UNKNOWN      39
Y           869
Name: fatal_(y/n), dtype: int64

In [86]:
sharks_attacks6[sharks_attacks6["fatal_(y/n)"] == "M"]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year
285,2016.04.18.b,18-Apr-2016,2016,Provoked,FRENCH POLYNESIA,Tuamotos,Makemo Atoll,Spearfishing,Hoata Iotua,Male,22.0,Laceration to knee by speared shark PROVOKED I...,M,Morning,"Grey reef shark, 2 m",2016-04-18,2016-04


In [87]:
sharks_attacks6["fatal_(y/n)"]= sharks_attacks6["fatal_(y/n)"].str.replace("UNKNOWN", "Unknown")
sharks_attacks6["fatal_(y/n)"]= sharks_attacks6["fatal_(y/n)"].str.replace("2017", "Y")
sharks_attacks6["fatal_(y/n)"]= sharks_attacks6["fatal_(y/n)"].str.replace("M", "Y")

In [88]:
shark_count_fatal = sharks_attacks6[["fatal_(y/n)"]]
shark_count_fatal.groupby(["fatal_(y/n)"])["fatal_(y/n)"].count()

fatal_(y/n)
N          3646
Unknown      39
Y           871
Name: fatal_(y/n), dtype: int64

In [89]:
sharks_attacks6.fillna(value={"fatal_(y/n)": "Unknown"}, inplace=True)

#### Let's check if we can find Unknown values of "fatal_(y/n) " in "injurity"

In [90]:
# We filter out rows in which "injury" values equal "FATAL" and fatal_(y/n)" is Unknown
sharks_attacks6[(sharks_attacks6["injury"] == "FATAL") & (sharks_attacks6["fatal_(y/n)"] == "Unknown")]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year
76,2017.10.01,01-Oct-2017,2017,Invalid,SOUTH AFRICA,Western Cape Province,Dyer Island,Scuba Diving,Bradley Fick,Male,31.0,FATAL,Unknown,NaN,Death may have been due to drowning,2017-10-01,2017-10
874,2011.08.28.b,28-Aug-2011,2011,Invalid,AUSTRALIA,Queensland,Fantome Island,Swimming,Rooster,Male,48.0,FATAL,Unknown,19:30,Shark involvement prior to death not confirmed,2011-08-28,2011-08
2230,1997.08.14.b,14-Aug-1997,1997,Invalid,MEXICO,Quintana Roo,"Santa Rosa, Cozumel",SCUBA diving,Mike Jonatis,Male,28.0,FATAL,Unknown,NaN,Shark involvement prior to death was not confi...,1997-08-14,1997-08
2660,1989.10.08,08-Oct-1989,1989,Invalid,USA,North Carolina,"Between Wrightsville Beach & Carolina Beach, N...",Diving,Doug Nunnally,Male,49.0,FATAL,Unknown,Late afternoon,Shark involvement suspected but not confirmed,1989-10-08,1989-10
2674,1989.07.27,27-Jul-1989,1989,Invalid,BERMUDA,NaN,NaN,Scuba diving,Russian male,Male,35.0,FATAL,Unknown,NaN,Shark involvement suspected but not confirmed,1989-07-27,1989-07
2953,1983.01.15,15-Jan-1983,1983,Invalid,SOUTH AFRICA,KwaZulu-Natal,Amanzimtoti,NaN,black male,Male,NaN,FATAL,Unknown,NaN,Shark involvement prior to death was not confi...,1983-01-15,1983-01
3811,1962.06.25,25-Jun-1962,1962,Invalid,USA,Florida,"Fernandina Beach, Nassau County",U.S. Airforce crewman reported missing after b...,male,Male,NaN,FATAL,Unknown,NaN,Shark involvement not confirmed,1962-06-25,1962-06
5196,1923.11.02,02-Nov-1923,1923,Invalid,AUSTRALIA,New South Wales,Bellinger Head,Fishing,male,Male,NaN,FATAL,Unknown,NaN,Shark involvement suspected but not confirmed,1923-11-02,1923-11
5425,1909.01.17,17-Jan-1909,1909,Invalid,NaN,NaN,Near the equator,Jumped overboard,Thomas Butler,Male,36.0,FATAL,Unknown,NaN,Questionable,1909-01-17,1909-01
5499,1905.03.25,25-Mar-1905,1905,Invalid,AUSTRALIA,Tasmania,Bridport,Bathing,Charles Taylor,Unknown,NaN,FATAL,Unknown,NaN,Questionable,1905-03-25,1905-03


#### Change "fatal" column to category

In [91]:
sharks_attacks5["fatal_(y/n)"] = sharks_attacks5["fatal_(y/n)"].astype("category")

#### We are going to replace here Unknown values from "fatal_(y/n)" for a "Y" whenever "injury" values is iqual to "FATAL"

In [92]:
sharks_attacks6.loc[sharks_attacks5["injury"] == "FATAL", "fatal_(y/n)"] = "Y"

#### We cheack if it works

In [93]:
sharks_attacks6[(sharks_attacks6["injury"] == "FATAL") & (sharks_attacks6["fatal_(y/n)"] != "Y")]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year


In [94]:
sharks_attacks6[(sharks_attacks6["injury"] == "FATAL") & (sharks_attacks6["fatal_(y/n)"] == "Unknown")]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year


### 4.8 Column species

In [95]:
sharks_attacks6["species"].unique()

array(['White shark', nan, '2 m shark', ...,
       'Fishermen recovered partial remains from shark a week later',
       "1.8 m to 2.7 m [6' to 9'] shark", 'Tiger shark, 3.9 m'],
      dtype=object)

#### Checking how many register we have of each category

In [96]:
shark_count_species = sharks_attacks6[["species"]]
shark_count_species.groupby(["species"])["species"].count()

species
                                     7
"A long thin brown-colored shark"    1
"A pack of 6 sharks"                 1
"A small shark"                      3
"Blue nose shark"                    1
                                    ..
unknown, possibly a white shark      1
whaler shark                         1
white shark                          1
white shark, 15'                     1
small brown shark                  1
Name: species, Length: 1336, dtype: int64

In [97]:
dict(shark_count_species.species.value_counts())

{'White shark': 141,
 'Shark involvement prior to death was not confirmed': 99,
 'Shark involvement not confirmed': 79,
 'Invalid': 75,
 'Tiger shark': 56,
 'Bull shark': 40,
 "4' shark": 40,
 "6' shark": 37,
 "1.8 m [6'] shark": 34,
 'Shark involvement prior to death unconfirmed': 27,
 "1.5 m [5'] shark": 25,
 "3' shark": 25,
 "5' shark": 25,
 "1.2 m [4'] shark": 23,
 "4' to 5' shark": 23,
 '2 m shark': 22,
 'Questionable': 21,
 'No shark involvement': 21,
 'Questionable incident': 20,
 "3 m [10'] shark": 19,
 "3' to 4' shark": 18,
 'Wobbegong shark': 17,
 "2.4 m [8'] shark": 16,
 'Blacktip shark': 16,
 '3 m shark': 15,
 'Shark involvement prior to death not confirmed': 13,
 'Raggedtooth shark': 13,
 "3.7 m [12'] shark": 13,
 "1.2 m to 1.5 m [4' to 5'] shark": 13,
 'Blue shark': 12,
 'Mako shark': 12,
 'Bronze whaler shark': 12,
 '"a small shark"': 12,
 "7' shark": 12,
 "5 m [16.5'] white shark": 12,
 "6 m [20'] white shark": 11,
 '1.5 m shark': 11,
 "12' shark": 11,
 'Grey nurse shar

#### We are going to try to group most common species

In [98]:
top_sharks = ["Tiger Shark", "Bull Shark", "Wobbegong Shark", "Blacktip Shark", "Raggedtooth Shark", "Blue Shark", "Hammerhead Shark", "Bronze whaler", "Lemon Shark", "Nurse Shark", "Mako Shark", "White Shark"]

In [99]:
# /.*?[T|t]iger.*\w?

In [100]:
# Tiger Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[T|t]iger.*\w?", "Tiger Shark")
# Bull Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[B|b]ull.*\w?", "Bull Shark")
# Wobbegong Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[W|w]obbegong.*\w?", "Wobbegong Shark")
# Blacktip Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[B|b]lacktip.*\w?", "Blacktip Shark")
# Raggedtooth Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[R|r]aggedtooth.*\w?", "Raggedtooth Shark")
# Blue Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[B|b]lue.*\w?", "Blue Shark")
# Hammerhead
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[H|h]ammerhead.*\w?", "Hammerhead Shark")
# Bronze whaler
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[B|b]ronze.*\w?", "Bronze whaler")
# Lemon Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[L|l]emon.*\w?", "Lemon Shark")
# Nurse Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[N|n]urse.*\w?", "Nurse Shark")
# Mako Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[M|m]ako.*\w?", "Mako Shark")
# White Shark
sharks_attacks6["species"] = sharks_attacks6["species"].str.replace(".*?[W|w]hite.*\w?", "White Shark")


#### We've got fewer categories  and everything that is not a nan value will put it the "Other" catergoy

In [101]:
sharks_attacks6.loc[~sharks_attacks6["species"].isin(top_sharks) & sharks_attacks6["species"].notnull(), "species"] = "Other"

In [102]:
shark_count_species2 = sharks_attacks6[["species"]]


In [103]:
dict(shark_count_species2.species.value_counts())

{'Other': 1502,
 'White Shark': 604,
 'Tiger Shark': 250,
 'Bull Shark': 157,
 'Blacktip Shark': 96,
 'Nurse Shark': 86,
 'Bronze whaler': 67,
 'Blue Shark': 48,
 'Mako Shark': 46,
 'Wobbegong Shark': 43,
 'Raggedtooth Shark': 41,
 'Hammerhead Shark': 40,
 'Lemon Shark': 30}

#### For all the nan values with change its value to "Unknown"

In [104]:
sharks_attacks6.fillna(value={"species": "Unknown"}, inplace=True)

In [105]:
dict(shark_count_species2.species.value_counts())

{'Other': 1502,
 'White Shark': 604,
 'Tiger Shark': 250,
 'Bull Shark': 157,
 'Blacktip Shark': 96,
 'Nurse Shark': 86,
 'Bronze whaler': 67,
 'Blue Shark': 48,
 'Mako Shark': 46,
 'Wobbegong Shark': 43,
 'Raggedtooth Shark': 41,
 'Hammerhead Shark': 40,
 'Lemon Shark': 30}

In [106]:
sharks_attacks6[sharks_attacks6["species"].isnull()]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year


In [107]:
# Cast types to save memory.
sharks_attacks6["species"] = sharks_attacks6["species"].astype("category")

In [108]:
# Checkpoint
sharks_attacks7 = sharks_attacks6

In [109]:
sharks_attacks7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   case_number  4967 non-null   object        
 1   date         4967 non-null   object        
 2   year         4967 non-null   int64         
 3   type         4967 non-null   category      
 4   country      4954 non-null   object        
 5   area         4733 non-null   object        
 6   location     4697 non-null   object        
 7   activity     4630 non-null   object        
 8   name         4839 non-null   object        
 9   sex          4967 non-null   category      
 10  age          3169 non-null   float64       
 11  injury       4953 non-null   object        
 12  fatal_(y/n)  4967 non-null   category      
 13  time         2779 non-null   object        
 14  species      4967 non-null   category      
 15  new_date     4967 non-null   datetime64[ns]
 16  month_

## 4.9 Column activity

#### Checking how many register we have of each category

In [110]:
sharks_attacks7.activity.unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       'Dangling feet in the water', 'Hunting seashells',
       'Standing, gathering oysters'], dtype=object)

In [111]:
shark_count_activity = sharks_attacks7[["activity"]]
shark_count_activity.groupby(["activity"])["activity"].count()
dict(shark_count_activity.activity.value_counts())

{'Surfing': 944,
 'Swimming': 746,
 'Fishing': 346,
 'Spearfishing': 273,
 'Wading': 136,
 'Standing': 87,
 'Diving': 86,
 'Snorkeling': 83,
 'Scuba diving': 70,
 'Bathing': 66,
 'Body boarding': 62,
 'Body surfing': 46,
 'Kayaking': 35,
 'Treading water': 29,
 'Boogie boarding': 27,
 'Free diving': 26,
 'Pearl diving': 20,
 'Windsurfing': 18,
 'Surf skiing': 18,
 'Walking': 16,
 'Boogie Boarding': 16,
 'Shark fishing': 13,
 'Floating': 13,
 'Rowing': 12,
 'Kayak Fishing': 11,
 'Surf fishing': 11,
 'Scuba Diving': 10,
 'Surf-skiing': 10,
 'Canoeing': 10,
 'Fell overboard': 9,
 'Fishing for sharks': 9,
 'Sitting on surfboard': 9,
 'Playing': 8,
 'Paddle boarding': 8,
 'Freediving': 8,
 'Diving for trochus': 8,
 'Surfing (sitting on his board)': 7,
 'Free diving for abalone': 7,
 'Diving for abalone': 7,
 'Sea disaster': 7,
 'Stand-Up Paddleboarding': 6,
 'Kite Surfing': 6,
 'Splashing': 5,
 'Freedom swimming': 5,
 'Skindiving': 5,
 'Sailing': 5,
 'Spearfishing on Scuba': 5,
 'Kite surfi

In [112]:
# Surfing
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[S|s]urfing.*\w?", "Surfing")
# Swimming
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[S|s]wimming.*\w?", "Swimming")
# Fishing
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[F|f]ishing.*\w?", "Fishing")
# Spearfishing
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[S|s]pearfishing.*\w?", "Spearfishing")
# Wading
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[W|w]ading.*\w?", "Wading")
# Standing
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[S|s]anding.*\w?", "Standing")
# Diving
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[D|d]iving.*\w?", "Diving")
# Snorkeling
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[S|s]norkeling.*\w?", "Snorkeling")
# Scuba diving
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[S|s]cuba.*\w?", "Scuba diving")
# Bathing
sharks_attacks7["activity"] = sharks_attacks7["activity"].str.replace(".*?[B|b]athing.*\w?", "Bathing")

In [113]:
shark_count_activity = sharks_attacks7[["activity"]]
shark_count_activity.groupby(["activity"])["activity"].count()
dict(shark_count_activity.activity.value_counts())

{'Surfing': 1087,
 'Fishing': 920,
 'Swimming': 888,
 'Diving': 404,
 'Wading': 149,
 'Snorkeling': 89,
 'Standing': 87,
 'Bathing': 78,
 'Body boarding': 62,
 'Kayaking': 35,
 'Treading water': 29,
 'Boogie boarding': 27,
 'Surf skiing': 18,
 'Walking': 16,
 'Boogie Boarding': 16,
 'Floating': 13,
 'Rowing': 12,
 'Surf-skiing': 10,
 'Canoeing': 10,
 'Sitting on surfboard': 9,
 'Fell overboard': 9,
 'Playing': 8,
 'Paddle boarding': 8,
 'Sea disaster': 7,
 'Stand-Up Paddleboarding': 6,
 'Sailing': 5,
 'Sea Disaster': 5,
 'Splashing': 5,
 'Body-boarding': 4,
 'Tagging sharks': 4,
 'Floating on his back': 4,
 'Jumping': 4,
 'Jumped into the water': 4,
 'Lifesaving drill': 4,
 'Shark watching': 3,
 'Paddling on surfboard': 3,
 'Feeding fish': 3,
 'Crabbing': 3,
 'Seine netting': 3,
 'Spearing fish': 3,
 'Playing in the surf': 3,
 'Their 9 m launch was run down by a 25,000-ton Japanese freighter  on the night of 3-11-1977 & they drifted, clinging to an icebox for 2 days': 3,
 'Boating': 3,

In [114]:
top_activities = ["Surfing","Fishing","Swimming", "Spearfishing","Wading","Standing","Diving","Snorkeling", "Scuba diving","Bathing"]

In [115]:
sharks_attacks7.loc[~sharks_attacks7["activity"].isin(top_activities ) & sharks_attacks7["activity"].notnull(), "activity"] = "Other"

#### For all the nan values with change its value to "Unknown

In [116]:
sharks_attacks7.fillna(value={"activity": "Unknown"}, inplace=True)

In [117]:
sharks_attacks7[sharks_attacks6["activity"].isnull()]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,new_date,month_year


In [118]:
# Cast types to save memory.
sharks_attacks7["activity"] = sharks_attacks6["activity"].astype("category")

In [119]:
sharks_attacks7["activity"].unique()

['Other', 'Standing', 'Surfing', 'Diving', 'Swimming', 'Fishing', 'Wading', 'Snorkeling', 'Unknown', 'Bathing']
Categories (10, object): ['Bathing', 'Diving', 'Fishing', 'Other', ..., 'Surfing', 'Swimming', 'Unknown', 'Wading']

In [120]:
shark_count_activity = sharks_attacks7[["activity"]]
shark_count_activity.groupby(["activity"])["activity"].count()
dict(shark_count_activity.activity.value_counts())

{'Surfing': 1087,
 'Other': 928,
 'Fishing': 920,
 'Swimming': 888,
 'Diving': 404,
 'Unknown': 337,
 'Wading': 149,
 'Snorkeling': 89,
 'Standing': 87,
 'Bathing': 78}

In [121]:
sharks_attacks7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   case_number  4967 non-null   object        
 1   date         4967 non-null   object        
 2   year         4967 non-null   int64         
 3   type         4967 non-null   category      
 4   country      4954 non-null   object        
 5   area         4733 non-null   object        
 6   location     4697 non-null   object        
 7   activity     4967 non-null   category      
 8   name         4839 non-null   object        
 9   sex          4967 non-null   category      
 10  age          3169 non-null   float64       
 11  injury       4953 non-null   object        
 12  fatal_(y/n)  4967 non-null   category      
 13  time         2779 non-null   object        
 14  species      4967 non-null   category      
 15  new_date     4967 non-null   datetime64[ns]
 16  month_

## 4.10 Column country

#### Checking how many register we have of each category

In [122]:
sharks_attacks7.country.unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'PUERTO RICO', 'ITALY',
       'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'FIJI', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA',
       'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'UNITE

In [123]:
# Checkpoint
sharks_attacks_final = sharks_attacks7

In [124]:
sharks_attacks_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4967 entries, 0 to 5561
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   case_number  4967 non-null   object        
 1   date         4967 non-null   object        
 2   year         4967 non-null   int64         
 3   type         4967 non-null   category      
 4   country      4954 non-null   object        
 5   area         4733 non-null   object        
 6   location     4697 non-null   object        
 7   activity     4967 non-null   category      
 8   name         4839 non-null   object        
 9   sex          4967 non-null   category      
 10  age          3169 non-null   float64       
 11  injury       4953 non-null   object        
 12  fatal_(y/n)  4967 non-null   category      
 13  time         2779 non-null   object        
 14  species      4967 non-null   category      
 15  new_date     4967 non-null   datetime64[ns]
 16  month_

## 5. Export dataframes into csv

In [125]:
sharks_attacks_final.to_csv('sharks_attacks_final.csv', sep =',',index = False)